In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install keras

In [3]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.0 MB/s eta 0:00:0000:010:01
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [4]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Layer
from keras.callbacks import LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense, Input, concatenate
from transformers import TFGPT2Model, GPT2Tokenizer
from keras.layers import GlobalAveragePooling1D
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, EarlyStopping

#### Data Loading and Preprocessing:
- Reads training and validation data from preprocessed files.
- Splits data into input sequences and labels.
- Performs tokenization and padding.


In [5]:
X_train, y_train = [], []
X_val, y_val = [], []
def read_data(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            for line in lines:
                # Split the line into word and label using tab delimiter
                parts = line.strip().split('\t')
                if len(parts) == 2:  # Ensure both word and label are present
                    word, label = parts
                    if file_path.endswith('preprocessed_data.txt'):  # Assuming training data file ends with 'train.txt'
                        X_train.append(word)
                        y_train.append(label)
                    elif file_path.endswith('preprocessed_val_data.txt'):  # Assuming validation data file ends with 'val.txt'
                        X_val.append(word)
                        y_val.append(label)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

# Load training data
read_data('/kaggle/input/corpus/preprocessed_data.txt')

# Load validation data
read_data('/kaggle/input/corpus/preprocessed_val_data.txt') 

print(len(X_train))
print(len(y_train))


50054
50054


In [13]:
# Character-level embedding parameters
char_vocab_size = 256  # Assuming ASCII characters
char_embedding_dim = 50

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train + X_val)  # Fit tokenizer on both training and validation data
X_train_indices = tokenizer.texts_to_sequences(X_train)
X_val_indices = tokenizer.texts_to_sequences(X_val)

# Padding sequences
max_sequence_length = max(
    max(len(seq) for seq in X_train_indices if seq),  # Ensure seq is not empty
    max(len(seq) for seq in X_val_indices if seq)      # Ensure seq is not empty
) if X_train_indices and X_val_indices else 0  # Check if any sequences exist
X_train_padded = pad_sequences(X_train_indices, maxlen=max_sequence_length)
X_val_padded = pad_sequences(X_val_indices, maxlen=max_sequence_length)

label_encoder = LabelEncoder()
label_encoder.fit(y_train + y_val)  # Fit label encoder on both training and validation labels
y_train_indices = label_encoder.transform(y_train)
y_val_indices = label_encoder.transform(y_val)

# Model parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
hidden_units = 64
num_classes = len(np.unique(y_train + y_val))
initial_lr = 0.001

#### Character-Level and Word-Level Embeddings:
- Defines parameters for character and word embeddings.
- Learns word embeddings using Keras Tokenizer.
- Concatenates character and word embeddings.


In [7]:
# Character input
char_input = Input(shape=(max_sequence_length,), dtype='int32')
char_embedding = Embedding(input_dim=char_vocab_size, output_dim=char_embedding_dim, input_length=max_sequence_length)(char_input)

# Word input
word_input = Input(shape=(max_sequence_length,), dtype='int32')
word_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length)(word_input)

# Concatenate character and word embeddings
concatenated_embeddings = concatenate([char_embedding, word_embedding])

# Load pre-trained GPT-2 model and tokenizer
gpt_model = TFGPT2Model.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  returned by default.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

#### GPT-2 Model Integration:
- Loads pre-trained GPT-2 model and tokenizer.
- Defines a custom layer for GPT-2 integration.
- Incorporates GPT-2 output into the model architecture.


In [8]:
class GPT2Layer(Layer):
    def __init__(self, gpt_model, **kwargs):
        super(GPT2Layer, self).__init__(**kwargs)
        self.gpt_model = gpt_model

    def call(self, inputs, **kwargs):
        return self.gpt_model(inputs)[0]  # Return the output of the GPT-2 model

input_layer = Input(shape=(max_sequence_length,), dtype='int32')
gpt_output = GPT2Layer(gpt_model)(input_layer)  # Use the custom GPT-2 layer
pooled_output = GlobalAveragePooling1D()(gpt_output)  # Reduce sequence dimension
output = Dense(num_classes, activation='softmax')(pooled_output)

#### Model Compilation and Training:
- Defines model architecture, compiles, and trains it.
- Implements learning rate scheduling and early stopping.


In [15]:
# Define learning rate schedule using ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

# Add Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Define model
model = Model(inputs=input_layer, outputs=output)

model.compile(optimizer=Adam(learning_rate=initial_lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_indices, validation_data=(X_val_padded, y_val_indices), epochs=10, batch_size=32, callbacks=[lr_scheduler, early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_val_padded, y_val_indices)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')


Epoch 1/10
   5/1565 ━━━━━━━━━━━━━━━━━━━━ 43s 28ms/step - accuracy: 0.0000e+00 - loss: 10.6979  

I0000 00:00:1716056034.138300     121 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1716056034.174775     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1565/1565 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6901 - loss: 2.0092

W0000 00:00:1716056063.047253     123 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1716056068.438057     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1565/1565 ━━━━━━━━━━━━━━━━━━━━ 61s 28ms/step - accuracy: 0.6901 - loss: 2.0090 - val_accuracy: 0.7230 - val_loss: 1.6386 - learning_rate: 0.0010
Epoch 2/10
   5/1565 ━━━━━━━━━━━━━━━━━━━━ 42s 27ms/step - accuracy: 0.7468 - loss: 1.5728

W0000 00:00:1716056078.355255     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1565/1565 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.7332 - loss: 1.5838 - val_accuracy: 0.7230 - val_loss: 1.5962 - learning_rate: 0.0010
Epoch 3/10
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.7380 - loss: 1.5409 - val_accuracy: 0.7231 - val_loss: 1.6147 - learning_rate: 0.0010
Epoch 4/10
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 31s 20ms/step - accuracy: 0.7351 - loss: 1.5386 - val_accuracy: 0.7230 - val_loss: 1.5944 - learning_rate: 0.0010
Epoch 5/10
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 31s 20ms/step - accuracy: 0.7362 - loss: 1.5264 - val_accuracy: 0.7230 - val_loss: 1.6342 - learning_rate: 0.0010
Epoch 6/10
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 31s 20ms/step - accuracy: 0.7389 - loss: 1.5043 - val_accuracy: 0.7232 - val_loss: 1.5731 - learning_rate: 0.0010
Epoch 7/10
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 31s 20ms/step - accuracy: 0.7342 - loss: 1.5104 - val_accuracy: 0.7230 - val_loss: 1.5557 - learning_rate: 0.0010
Epoch 8/10
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 31s 20ms/step - accuracy: 0.7356 

#### Evaluation:
- Evaluates trained model on validation data.
- Calculates and prints loss and accuracy.

In [19]:
# Predict probabilities for each class
y_pred_probs = model.predict(X_val_padded)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Get the unique classes in the validation data
unique_labels = np.unique(y_val_indices)

# Calculate all metrics
report = classification_report(y_val_indices, y_pred, labels=unique_labels, target_names=label_encoder.classes_[unique_labels], output_dict=True)

# Access individual metrics
accuracy = report['accuracy']
precision = report['weighted avg']['precision']
recall = report['weighted avg']['recall']
f1_score = report['weighted avg']['f1-score']

print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1_score}')

456/456 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step
Accuracy: 0.7231064763995609, Precision: 0.525359620466021, Recall: 0.7231064763995609, F1 Score: 0.6069982139565608


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
